# Finding the Mass of a W-boson from Four-momentum

I am hoping for the neural network to learn the formula $$M^2 = E^2 - p^2$$.

To do this, I will be using a neural network with 4 input nodes (px, py, pz, E), 1 hidden layer with 3 hidden nodes (as per these [guidelines](https://stats.stackexchange.com/questions/181/how-to-choose-the-number-of-hidden-layers-and-nodes-in-a-feedforward)), and 1 output node (the mass of the W-boson)

As for the intial weights and biases, I will be using random numbers from -1 to 1 for weights. All biases will be initially 0.

I will also be using the sigmoid activation function and calculating error using MSE.

I will start at 3 epochs, and adjust this value as neceesary.

Finally, the learning rate will be 0.1, and the momentum term will be 0.9 (based on [this article](https://machinelearningmastery.com/learning-rate-for-deep-learning-neural-networks/)).

Here is a table for the first set of weights (4 input nodes to 3 hidden nodes)

| Node 1 | Node 2 | Node 3 |
| --- | --- | --- |
| 0.0512580941 | 0.6848640862 | 0.1639099298 |
|-0.3910101941 | -0.8762026371 | 0.3900340812 |
|0.6318537830 | 0.1662571451 | -0.2801570261 |
|-0.8242067952 | -0.8138137694 | 0.3600363740 |

And here is a table for the second set of weights (3 hidden nodes to 1 output nodes)

| Node 1 |
| --- |
| 0.1617690173 |
| 0.0258649108 |
| -0.2123283647 |

First we will get the data from the 1TeVDecay file and input our weights, biases.

In [5]:
import pandas as pd
import math
import numpy as np
TeVData = pd.read_csv("1TeVDecay.csv")
px = np.asarray(TeVData[TeVData.PID == -24]['Px'])
py = np.asarray(TeVData[TeVData.PID == -24]['Py'])
pz = np.asarray(TeVData[TeVData.PID == -24]['Pz'])
e = np.asarray(TeVData[TeVData.PID == -24]['E'])
m = np.asarray(TeVData[TeVData.PID == -24]['M'])
b1 = [0, 0, 0, 0]
b1Old = [0, 0, 0, 0]
b2 = [0]
b2Old = [0]
w1 = [[0.0512580941, 0.6848640862, 0.1639099298, -0.132456], [-0.3910101941, -0.8762026371, 0.3900340812, 0.847204], [0.6318537830, 0.1662571451, -0.2801570261, -0.3466891], [-0.8242067952, -0.8138137694, 0.3600363740, 0.37169346]]
w1Old = [[0.0512580941, 0.6848640862, 0.1639099298, -0.132456], [-0.3910101941, -0.8762026371, 0.3900340812, 0.847204], [0.6318537830, 0.1662571451, -0.2801570261, -0.3466891], [-0.8242067952, -0.8138137694, 0.3600363740, 0.37169346]]
w2 = [[0.1617690173], [0.0258649108], [-0.2123283647], [0.12348712]]
w2Old = [[0.1617690173], [0.0258649108], [-0.2123283647], [0.12348712]]
learningRate = 0.1
momentumTerm = 0.9

[80.41900245 80.41900245 80.41900245 ... 80.41900245 80.41900245
 80.41900245]


Next, we will run each epoch of the training:

In [3]:
sumSquaredErrors = 0.0
for a in range(0, 3):
    for i in range(0, int(len(px) * .9)): #other 10% will be part of the testing dataset
        input = [px[i], py[i], pz[i], e[i]]
        target = [m[i] * m[i]]
        #forward propagation
        y1 = []
        for j in range(0, 4):
            v = 0.0
            for k in range(0, len(input)):
                v += w1[k][j] * input[k]
            v += b1[j]
            if(v >= 0):
                y1.append(v)
            else:
                y1.append(0)
        y2 = []
        for j in range(0, 1):
            v = 0.0
            for k in range(0, len(y1)):
                v += w2[k][j] * y1[k]
            v += b2[j]
            if(v >= 0):
                y2.append(v)
            else:
                y2.append(0)
        #calculating error
        squareError = (target[j] - y2[j]) ** 2.0
        sumSquaredErrors += squareError
        #backpropagation outer layer
        for j in range(0, 1):
            localGradient = (target[j] - y2[j]) * y2[j] * (1 - y2[j])
            #weights
            for k in range(0, 4):
                tempW = w2[k][j]
                w2[k][j] = w2[k][j] + momentumTerm * (w2[k][j] - w2Old[k][j]) + learningRate * localGradient * y1[k]
                w2Old[k][j] = tempW
            #biases
            tempB = b2[j]
            b2[j] = b2[j] + momentumTerm * (b2[j] - b2Old[j]) + learningRate * localGradient
            b2Old[j] = tempB
        #backpropagation hidden layer
        for j in range(0, 4):
            localGradient = y1[j] * (1 - y1[j])
            sigmaGradientWeight = 0.0
            for k in range(0, 1):
                sigmaGradientWeight += (target[k] - y2[k]) * y2[k] * (1 - y2[k]) * w2Old[j][k]
            localGradient *= sigmaGradientWeight
            #weights
            for k in range(0, 4):
                tempW = w1[k][j]
                w1[k][j] = w1[k][j] + momentumTerm * (w1[k][j] - w1Old[k][j]) + learningRate * localGradient * input[k]
                w1Old[k][j] = tempW
            #biases
            tempB = b1[j]
            b1[j] = b1[j] + momentumTerm * (b1[j] - b1Old[j]) + learningRate * localGradient
            b1Old[j] = tempB
        #output after 1 input
        print("Epoch " + str(a + 1) + " Training (" + str(i + 1) + " / " + str(int(len(px) * .9)) + "): " + str(squareError))
    print("--------------------------------------------------------------------------------------------------------")


Epoch 1 Training (1 / 4491): 41824882.20097205
Epoch 1 Training (2 / 4491): 41824882.20097205
Epoch 1 Training (3 / 4491): 41824882.20097205
Epoch 1 Training (4 / 4491): 41824882.20097205
Epoch 1 Training (5 / 4491): 41824882.20097205
Epoch 1 Training (6 / 4491): 41824882.20097205
Epoch 1 Training (7 / 4491): 41824882.20097205
Epoch 1 Training (8 / 4491): 41824882.20097205
Epoch 1 Training (9 / 4491): 41824882.20097205
Epoch 1 Training (10 / 4491): 41824882.20097205
Epoch 1 Training (11 / 4491): 41824882.20097205
Epoch 1 Training (12 / 4491): 41824882.20097205
Epoch 1 Training (13 / 4491): 41824882.20097205
Epoch 1 Training (14 / 4491): 41824882.20097205
Epoch 1 Training (15 / 4491): 41824882.20097205
Epoch 1 Training (16 / 4491): 41824882.20097205
Epoch 1 Training (17 / 4491): 41824882.20097205
Epoch 1 Training (18 / 4491): 41824882.20097205
Epoch 1 Training (19 / 4491): 41824882.20097205
Epoch 1 Training (20 / 4491): 41824882.20097205
Epoch 1 Training (21 / 4491): 41824882.20097205
E

Epoch 1 Training (1638 / 4491): 41824882.20097205
Epoch 1 Training (1639 / 4491): 41824882.20097205
Epoch 1 Training (1640 / 4491): 41824882.20097205
Epoch 1 Training (1641 / 4491): 41824882.20097205
Epoch 1 Training (1642 / 4491): 41824882.20097205
Epoch 1 Training (1643 / 4491): 41824882.20097205
Epoch 1 Training (1644 / 4491): 41824882.20097205
Epoch 1 Training (1645 / 4491): 41824882.20097205
Epoch 1 Training (1646 / 4491): 41824882.20097205
Epoch 1 Training (1647 / 4491): 41824882.20097205
Epoch 1 Training (1648 / 4491): 41824882.20097205
Epoch 1 Training (1649 / 4491): 41824882.20097205
Epoch 1 Training (1650 / 4491): 41824882.20097205
Epoch 1 Training (1651 / 4491): 41824882.20097205
Epoch 1 Training (1652 / 4491): 41824882.20097205
Epoch 1 Training (1653 / 4491): 41824882.20097205
Epoch 1 Training (1654 / 4491): 41824882.20097205
Epoch 1 Training (1655 / 4491): 41824882.20097205
Epoch 1 Training (1656 / 4491): 41824882.20097205
Epoch 1 Training (1657 / 4491): 41824882.20097205


Epoch 1 Training (4418 / 4491): 41824882.20097205
Epoch 1 Training (4419 / 4491): 41824882.20097205
Epoch 1 Training (4420 / 4491): 41824882.20097205
Epoch 1 Training (4421 / 4491): 41824882.20097205
Epoch 1 Training (4422 / 4491): 41824882.20097205
Epoch 1 Training (4423 / 4491): 41824882.20097205
Epoch 1 Training (4424 / 4491): 41824882.20097205
Epoch 1 Training (4425 / 4491): 41824882.20097205
Epoch 1 Training (4426 / 4491): 41824882.20097205
Epoch 1 Training (4427 / 4491): 41824882.20097205
Epoch 1 Training (4428 / 4491): 41824882.20097205
Epoch 1 Training (4429 / 4491): 41824882.20097205
Epoch 1 Training (4430 / 4491): 41824882.20097205
Epoch 1 Training (4431 / 4491): 41824882.20097205
Epoch 1 Training (4432 / 4491): 41824882.20097205
Epoch 1 Training (4433 / 4491): 41824882.20097205
Epoch 1 Training (4434 / 4491): 41824882.20097205
Epoch 1 Training (4435 / 4491): 41824882.20097205
Epoch 1 Training (4436 / 4491): 41824882.20097205
Epoch 1 Training (4437 / 4491): 41824882.20097205


Epoch 2 Training (1892 / 4491): 41824882.20097205
Epoch 2 Training (1893 / 4491): 41824882.20097205
Epoch 2 Training (1894 / 4491): 41824882.20097205
Epoch 2 Training (1895 / 4491): 41824882.20097205
Epoch 2 Training (1896 / 4491): 41824882.20097205
Epoch 2 Training (1897 / 4491): 41824882.20097205
Epoch 2 Training (1898 / 4491): 41824882.20097205
Epoch 2 Training (1899 / 4491): 41824882.20097205
Epoch 2 Training (1900 / 4491): 41824882.20097205
Epoch 2 Training (1901 / 4491): 41824882.20097205
Epoch 2 Training (1902 / 4491): 41824882.20097205
Epoch 2 Training (1903 / 4491): 41824882.20097205
Epoch 2 Training (1904 / 4491): 41824882.20097205
Epoch 2 Training (1905 / 4491): 41824882.20097205
Epoch 2 Training (1906 / 4491): 41824882.20097205
Epoch 2 Training (1907 / 4491): 41824882.20097205
Epoch 2 Training (1908 / 4491): 41824882.20097205
Epoch 2 Training (1909 / 4491): 41824882.20097205
Epoch 2 Training (1910 / 4491): 41824882.20097205
Epoch 2 Training (1911 / 4491): 41824882.20097205


Epoch 2 Training (3719 / 4491): 41824882.20097205
Epoch 2 Training (3720 / 4491): 41824882.20097205
Epoch 2 Training (3721 / 4491): 41824882.20097205
Epoch 2 Training (3722 / 4491): 41824882.20097205
Epoch 2 Training (3723 / 4491): 41824882.20097205
Epoch 2 Training (3724 / 4491): 41824882.20097205
Epoch 2 Training (3725 / 4491): 41824882.20097205
Epoch 2 Training (3726 / 4491): 41824882.20097205
Epoch 2 Training (3727 / 4491): 41824882.20097205
Epoch 2 Training (3728 / 4491): 41824882.20097205
Epoch 2 Training (3729 / 4491): 41824882.20097205
Epoch 2 Training (3730 / 4491): 41824882.20097205
Epoch 2 Training (3731 / 4491): 41824882.20097205
Epoch 2 Training (3732 / 4491): 41824882.20097205
Epoch 2 Training (3733 / 4491): 41824882.20097205
Epoch 2 Training (3734 / 4491): 41824882.20097205
Epoch 2 Training (3735 / 4491): 41824882.20097205
Epoch 2 Training (3736 / 4491): 41824882.20097205
Epoch 2 Training (3737 / 4491): 41824882.20097205
Epoch 2 Training (3738 / 4491): 41824882.20097205


Epoch 3 Training (1362 / 4491): 41824882.20097205
Epoch 3 Training (1363 / 4491): 41824882.20097205
Epoch 3 Training (1364 / 4491): 41824882.20097205
Epoch 3 Training (1365 / 4491): 41824882.20097205
Epoch 3 Training (1366 / 4491): 41824882.20097205
Epoch 3 Training (1367 / 4491): 41824882.20097205
Epoch 3 Training (1368 / 4491): 41824882.20097205
Epoch 3 Training (1369 / 4491): 41824882.20097205
Epoch 3 Training (1370 / 4491): 41824882.20097205
Epoch 3 Training (1371 / 4491): 41824882.20097205
Epoch 3 Training (1372 / 4491): 41824882.20097205
Epoch 3 Training (1373 / 4491): 41824882.20097205
Epoch 3 Training (1374 / 4491): 41824882.20097205
Epoch 3 Training (1375 / 4491): 41824882.20097205
Epoch 3 Training (1376 / 4491): 41824882.20097205
Epoch 3 Training (1377 / 4491): 41824882.20097205
Epoch 3 Training (1378 / 4491): 41824882.20097205
Epoch 3 Training (1379 / 4491): 41824882.20097205
Epoch 3 Training (1380 / 4491): 41824882.20097205
Epoch 3 Training (1381 / 4491): 41824882.20097205


Epoch 3 Training (3305 / 4491): 41824882.20097205
Epoch 3 Training (3306 / 4491): 41824882.20097205
Epoch 3 Training (3307 / 4491): 41824882.20097205
Epoch 3 Training (3308 / 4491): 41824882.20097205
Epoch 3 Training (3309 / 4491): 41824882.20097205
Epoch 3 Training (3310 / 4491): 41824882.20097205
Epoch 3 Training (3311 / 4491): 41824882.20097205
Epoch 3 Training (3312 / 4491): 41824882.20097205
Epoch 3 Training (3313 / 4491): 41824882.20097205
Epoch 3 Training (3314 / 4491): 41824882.20097205
Epoch 3 Training (3315 / 4491): 41824882.20097205
Epoch 3 Training (3316 / 4491): 41824882.20097205
Epoch 3 Training (3317 / 4491): 41824882.20097205
Epoch 3 Training (3318 / 4491): 41824882.20097205
Epoch 3 Training (3319 / 4491): 41824882.20097205
Epoch 3 Training (3320 / 4491): 41824882.20097205
Epoch 3 Training (3321 / 4491): 41824882.20097205
Epoch 3 Training (3322 / 4491): 41824882.20097205
Epoch 3 Training (3323 / 4491): 41824882.20097205
Epoch 3 Training (3324 / 4491): 41824882.20097205


In [4]:
print(y2)

[0]
